In [4]:
!pip install langgraph langchain openai

In [5]:
!pip install langchain langgraph python-dotenv openai

In [16]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from pydantic import BaseModel
from dotenv import load_dotenv
import os

# .env에서 API 키 불러오기
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY가 설정되지 않았습니다.")

# ChatOpenAI 모델 설정
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 상태 모델 정의
class CodeState(BaseModel):
    code: str
    style_feedback: str | None = None
    bug_feedback: str | None = None
    optimization_feedback: str | None = None
    final_code: str | None = None

# 각 피드백 노드 정의
def style_check_node(state: CodeState) -> CodeState:
    messages = [
        SystemMessage(content="당신은 코드 스타일 전문가입니다. PEP8 기준으로 코드를 점검하세요."),
        HumanMessage(content=f"다음 코드의 스타일을 점검해 주세요:\n\n{state.code}")
    ]
    feedback = llm(messages).content
    return state.copy(update={"style_feedback": feedback})

def bug_check_node(state: CodeState) -> CodeState:
    messages = [
        SystemMessage(content="당신은 버그 분석 전문가입니다."),
        HumanMessage(content=f"다음 코드의 버그 가능성을 분석해 주세요:\n\n{state.code}")
    ]
    feedback = llm(messages).content
    return state.copy(update={"bug_feedback": feedback})

def optimization_node(state: CodeState) -> CodeState:
    messages = [
        SystemMessage(content="당신은 성능 최적화 전문가입니다."),
        HumanMessage(content=f"다음 코드의 최적화 가능성을 분석해 주세요:\n\n{state.code}")
    ]
    feedback = llm(messages).content
    return state.copy(update={"optimization_feedback": feedback})

def refactor_node(state: CodeState) -> CodeState:
    messages = [
        SystemMessage(content="당신은 리팩터링 전문가입니다."),
        HumanMessage(content=f"""
기존 코드:
{state.code}

스타일 피드백:
{state.style_feedback}

버그 피드백:
{state.bug_feedback}

최적화 피드백:
{state.optimization_feedback}

위 피드백을 모두 반영해서 코드를 개선해 주세요.
""")
    ]
    result = llm(messages).content
    return state.copy(update={"final_code": result})

# LangGraph 워크플로우 구성
def build_graph():
    builder = StateGraph(CodeState)  # ✅ 상태는 CodeState 기반이어야 함
    builder.add_node("StyleCheck", style_check_node)
    builder.add_node("BugCheck", bug_check_node)
    builder.add_node("Optimize", optimization_node)
    builder.add_node("Refactor", refactor_node)

    builder.set_entry_point("StyleCheck")
    builder.add_edge("StyleCheck", "BugCheck")
    builder.add_edge("BugCheck", "Optimize")
    builder.add_edge("Optimize", "Refactor")
    builder.add_edge("Refactor", END)

    return builder.compile()

# 실행
if __name__ == "__main__":
    code_input = """
def add(x,y):return x+y
def divide(a,b): return a / b
"""
    app = build_graph()
    result = app.invoke(CodeState(code=code_input))  # ✅ BaseModel 객체로 전달

    print("\n🔍 스타일 피드백:\n", result.style_feedback)
    print("\n🐞 버그 피드백:\n", result.bug_feedback)
    print("\n🚀 최적화 피드백:\n", result.optimization_feedback)
    print("\n✅ 최종 개선 코드:\n", result.final_code)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-api-key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}